# Lab: 1 Data Preprocessing
####  In this lab, we will explore Data Preprocessing, a critical step in the data mining process that prepares raw data for analysis by cleaning, reducing, normalizing, and discretizing it.

- Student ID: 21127087  
- Student Name: Nguyen Gia Kiet
- Self-evaluation of the assignment requirements: 90%

### Import library

In [4]:
import pandas as pd
import numpy as np

### Read data
First we have to read data from csv file

In [5]:
columns = ['age', 'workclass', 'fnlwgt', 'education', 'education-num', 'marital-status', 'occupation', 'relationship', 'race', 'sex', 'capital-gain', 'capital-loss', 'hours-per-week', 'native-country', 'income']

df = pd.read_csv("adult.csv", header=None, names=columns)

# Strip whitespace from the column names
for col in df.columns:
    if df[col].dtype == 'object':
        df[col] = df[col].str.strip()
    else:
        df[col] = pd.to_numeric(df[col])

df.head()


,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


## Data cleaning

### Assessment of Missing Data

I check with (?) because missing data in this database is represented by the (?) symbol.

In [6]:
# print(df.isin(['?']).sum())
print(df.isin([0]).sum())

age                   0
workclass             0
fnlwgt                0
education             0
education-num         0
marital-status        0
occupation            0
relationship          0
race                  0
sex                   0
capital-gain      29849
capital-loss      31042
hours-per-week        0
native-country        0
income                0
dtype: int64


### Handle Missing Values

Based on the observation, the columns with missing data are as follows:

- workclass: 1,836 missing values
- occupation: 1,843 missing values
- native-country: 583 missing values

Since these columns are primarily categorical, we can impute the missing values using the mode (the most frequent value). Using the mode helps preserve the data distribution and avoids introducing noise into the original dataset.

- The mode is calculated by counting the frequency of each value in the column and selecting the value with the highest frequency:

$$ \text{Mode} = \arg\max_{x \in X} frequency(x) $$

where frequency(x) represents the number of occurrences of the value 𝑥.

In [7]:
df = df.replace('?', np.NaN)

for col in ['workclass', 'occupation', 'native-country']:
    df[col] = df[col].fillna(df[col].mode()[0])

print(df.isna().sum())

age               0
workclass         0
fnlwgt            0
education         0
education-num     0
marital-status    0
occupation        0
relationship      0
race              0
sex               0
capital-gain      0
capital-loss      0
hours-per-week    0
native-country    0
income            0
dtype: int64


### Check and Handle Duplicate Data

To ensure the quality of the dataset, we need to check for duplicate entries and handle them appropriately. Duplicate rows can skew the analysis and lead to overfitting in machine learning models.

- **Step 1: Identify Duplicates**  
  We will check the dataset for any duplicate rows by comparing all columns. If duplicates are found, they will be reported along with the total count.

In [8]:
print("Duplicated : ",df.duplicated().sum())

df = df.drop_duplicates()

Duplicated :  24



- **Step 2: Handle Duplicates**  
  If duplicates exist, we will remove them to avoid redundancy. Removing duplicates ensures that each data point is unique, preserving the integrity of the dataset for further analysis or modeling.

In [9]:
print("Dropped: ",df.duplicated().sum())

Dropped:  0


### Feature Selection

#### Analyze Correlation

To identify the most relevant numerical features for predicting the target variable (`income`), we analyze the Pearson correlation coefficient between numerical columns and `income`. This step helps us understand which features have a strong linear relationship with the target, allowing us to prioritize features with higher correlation and potentially remove those with low correlation.

- **Step 1: Convert the Target Variable**  
  The target variable `income` is categorical (`<=50K` and `>50K`). We convert it to numerical values (`0` for `<=50K` and `1` for `>50K`) to compute the correlation with numerical features.

In [10]:
df['income'] = df['income'].map({'<=50K': 0, '>50K': 1})

- **Step 2: Compute Pearson Correlation**  
  We calculate the Pearson correlation coefficient between each numerical feature (`age`, `fnlwgt`, `education-num`, `capital-gain`, `capital-loss`, `hours-per-week`) and the target `income`. Features with higher absolute correlation values are more likely to be useful for predicting `income`.

In [11]:
# Step 2: Define a function to calculate Pearson Correlation manually
def pearson_correlation(x, y):
    x_mean = np.mean(x)
    y_mean = np.mean(y)
    numerator = np.sum((x - x_mean) * (y - y_mean))
    denominator = np.sqrt(np.sum((x - x_mean)**2) * np.sum((y - y_mean)**2))
    return numerator / denominator if denominator != 0 else 0

# List of numerical columns
numeric_cols = ['age', 'fnlwgt', 'education-num', 'capital-gain', 'capital-loss', 'hours-per-week']

# Calculate correlation between each numerical column and 'income'
correlations = {}
for col in numeric_cols:
    corr = pearson_correlation(df[col], df['income'])
    correlations[col] = corr

# Sort correlations in descending order
sorted_correlations = sorted(correlations.items(), key=lambda x: x[1], reverse=True)

# Display the correlation coefficients
print("Correlation coefficients with 'income' for numerical columns:")
for col, corr in sorted_correlations:
    print(f"{col}: {corr:.4f}")

Correlation coefficients with 'income' for numerical columns:
education-num: 0.3353
age: 0.2340
hours-per-week: 0.2297
capital-gain: 0.2233
capital-loss: 0.1505
fnlwgt: -0.0095


#### Analyze Categorical Features Using Information Gain

After analyzing the correlation for numerical features, we now evaluate the importance of categorical features by calculating their **Information Gain (IG)** with respect to the target variable `income`. This metric helps us measure how much uncertainty in `income` is reduced by knowing the value of a categorical feature, allowing us to prioritize features that are most predictive of the target.

- **Step 1: Understand Information Gain**  
  Information Gain is calculated using the following formula:

  $$
  \text{IG} = \text{Entropy(Total)} - \sum_{v \in \text{values}} \frac{|D_v|}{|D|} \cdot \text{Entropy}(D_v)
  $$

  **Explanation of Terms:**  
  - **IG**: Information Gain, a measure of the reduction in uncertainty.  
  - **Entropy(Total)**: The entropy of the entire dataset.  
  - **$\sum_{v \in \text{values}}$**: Summation over all possible values \( v \) of the feature.  
  - **$\frac{|D_v|}{|D|}$**: The proportion of the subset \( D_v \) in the dataset \( D \).  
  - **Entropy($D_v$)**: The entropy of the subset \( D_v \).

- **Step 2: Compute Information Gain for Categorical Features**  
  We will calculate the Information Gain for each categorical feature (`workclass`, `education`, `marital-status`, `occupation`, `relationship`, `race`, `sex`, `native-country`) with respect to `income`. Features with higher IG values are more informative and will be prioritized for modeling.

Below is the Python code to compute Information Gain for categorical features:


In [12]:
# Function to calculate Information Gain
def calculate_information_gain(df, feature, target):
    # Calculate the total entropy of the dataset
    total_entropy = -sum(df[target].value_counts(normalize=True) * 
                         np.log2(df[target].value_counts(normalize=True) + 1e-10))
    
    # Calculate the weighted entropy for each value of the feature
    weighted_entropy = 0
    for value in df[feature].unique():
        subset = df[df[feature] == value]
        subset_entropy = -sum(subset[target].value_counts(normalize=True) * 
                              np.log2(subset[target].value_counts(normalize=True) + 1e-10))
        weighted_entropy += (len(subset) / len(df)) * subset_entropy
    
    # Information Gain = Total Entropy - Weighted Entropy
    return total_entropy - weighted_entropy

# List of categorical columns
categorical_cols = ['workclass', 'education', 'marital-status', 'occupation', 
                    'relationship', 'race', 'sex', 'native-country']

# Calculate Information Gain for each categorical column
importance = {}
for col in categorical_cols:
    ig = calculate_information_gain(df, col, 'income')
    importance[col] = ig

# Sort features by Information Gain in descending order
sorted_importance = sorted(importance.items(), key=lambda x: x[1], reverse=True)

# Display the Information Gain values
print("Information Gain for categorical columns (sorted from highest to lowest):")
for col, ig in sorted_importance:
    print(f"{col}: {ig:.4f}")

Information Gain for categorical columns (sorted from highest to lowest):
relationship: 0.1653
marital-status: 0.1565
education: 0.0936
occupation: 0.0759
sex: 0.0372
workclass: 0.0181
native-country: 0.0086
race: 0.0084


#### Correlation Results for Numerical Features (Pearson)

| Feature          | Pearson Correlation |
|------------------|---------------------|
| `education-num`  | 0.3353             |
| `age`            | 0.2340             |
| `hours-per-week` | 0.2297             |
| `capital-gain`   | 0.2233             |
| `capital-loss`   | 0.1505             |
| `fnlwgt`         | -0.0095            |

#### Importance of Categorical Features (Information Gain)

| Feature          | Information Gain |
|------------------|------------------|
| `relationship`   | 0.1653          |
| `marital-status` | 0.1565          |
| `education`      | 0.0936          |
| `occupation`     | 0.0759          |
| `sex`            | 0.0372          |
| `workclass`      | 0.0181          |
| `native-country` | 0.0086          |
| `race`           | 0.0084          |

#### Feature Selection Decision

Based on the correlation and Information Gain results, the selected features are:  
`age`, `education-num`, `relationship`, `occupation`, `capital-gain`, `capital-loss`, `hours-per-week`.

**Reasoning for Feature Selection:**

- **`education-num` (Pearson: 0.3353)**: This feature has the highest Pearson correlation with `income`, indicating a strong relationship. We remove the `education` column (IG: 0.0936) since it conveys similar information and is directly mapped to `education-num`.
- **`age` (Pearson: 0.2340)**: This feature has a high correlation with `income`. Age is often related to experience, which can influence income levels.
- **`relationship` (IG: 0.1653)**: This feature has the highest Information Gain among categorical features, reflecting how a person's role in the family (e.g., responsibility) impacts their financial status. We remove `marital-status` (IG: 0.1565), despite its high IG, because it is highly correlated with `relationship` (to be confirmed by the Chi-squared test and Cramér's V).
- **`occupation` (IG: 0.0759)**: This feature has a sufficiently high IG value, and occupation is a key determinant of income.
- **`capital-gain` (Pearson: 0.2233)**: Capital gain is a direct indicator of wealth and has a high correlation with `income`.
- **`capital-loss` (Pearson: 0.1505)**: Capital loss provides additional financial information and has a reasonable correlation with `income`.
- **`hours-per-week` (Pearson: 0.2297)**: The number of hours worked per week is related to productivity and income, with a high correlation.


In [13]:

selected_features = ['age', 'education-num', 'relationship', 'occupation', 
                     'capital-gain', 'capital-loss', 'hours-per-week', 'income']
df = df[selected_features]

print(f"Number of feature selection: {len(selected_features) - 1}")
print("Updated data:")
df.head(10)

Number of feature selection: 7
Updated data:


,age,education-num,relationship,occupation,capital-gain,capital-loss,hours-per-week,income
0,39,13,Not-in-family,Adm-clerical,2174,0,40,0
1,50,13,Husband,Exec-managerial,0,0,13,0
2,38,9,Not-in-family,Handlers-cleaners,0,0,40,0
3,53,7,Husband,Handlers-cleaners,0,0,40,0
4,28,13,Wife,Prof-specialty,0,0,40,0
5,37,14,Wife,Exec-managerial,0,0,40,0
6,49,5,Not-in-family,Other-service,0,0,16,0
7,52,9,Husband,Exec-managerial,0,0,45,1
8,31,14,Not-in-family,Prof-specialty,14084,0,50,1
9,42,13,Husband,Exec-managerial,5178,0,40,1


### Data Normalization

After selecting the features, the numerical attributes need to be normalized due to their varying ranges:

- **`age`**: Ranges from 17 to 90.
- **`education-num`**: Ranges from 1 to 16.
- **`capital-gain`**: Ranges from 0 to 99,999 (highly skewed with many 0 values).
- **`capital-loss`**: Ranges from 0 to 4,356 (highly skewed with many 0 values).
- **`hours-per-week`**: Ranges from 1 to 99.

These columns have significantly different ranges, which can bias the scale of machine learning algorithms. For example, the large values in `capital-gain` may dominate smaller values in `age`, leading to poor model performance.

To address this, we will use **Min-Max Scaling** to normalize the data into the range \([0, 1]\). However, since `capital-gain` and `capital-loss` have a highly skewed distribution with many 0 values, applying Min-Max Scaling alone may not be effective. To mitigate the skewness, we will first apply **Log Transformation** before Min-Max Scaling.

#### Min-Max Scaling

The formula for Min-Max Scaling is as follows:

$$
X_{\text{scaled}} = \frac{X - X_{\text{min}}}{X_{\text{max}} - X_{\text{min}}}
$$

**Explanation of Terms:**  
- $X$: The original value.  
- $X_{\text{min}}, X_{\text{max}}$: The minimum and maximum values in the column.  
- $X_{\text{scaled}}$: The normalized value, which lies in the range \([0, 1]\).

#### Log Transformation

To reduce the skewness in `capital-gain` and `capital-loss`, we apply Log Transformation before Min-Max Scaling. The formula for Log Transformation is:

$$
X_{\log} = \log(X + 1)
$$

- We add 1 to \(X\) to avoid the error \(\log(0)\), since many values in `capital-gain` and `capital-loss` are 0.

In [14]:
# Define the Min-Max Scaling function
def min_max_scaling(column):
    min_val = column.min()
    max_val = column.max()
    if max_val == min_val:
        return column * 0
    return (column - min_val) / (max_val - min_val)

# List of numerical columns to normalize
numeric_cols = ['age', 'education-num', 'capital-gain', 'capital-loss', 'hours-per-week']

# Apply Log Transformation and Min-Max Scaling
for col in numeric_cols:
    if col in ['capital-gain', 'capital-loss']:
        print(f"Applying Log Transformation to {col} to reduce skewness.")
        df[col] = np.log1p(df[col])  # log1p = log(1 + x)
    df[col] = min_max_scaling(df[col])

# Display the normalized data
print("\nData after Log Transformation + Min-Max Scaling:")
print(df[numeric_cols].head(10))

Applying Log Transformation to capital-gain to reduce skewness.
Applying Log Transformation to capital-loss to reduce skewness.

Data after Log Transformation + Min-Max Scaling:
        age  education-num  capital-gain  capital-loss  hours-per-week
0  0.301370       0.800000      0.667492           0.0        0.397959
1  0.452055       0.800000      0.000000           0.0        0.122449
2  0.287671       0.533333      0.000000           0.0        0.397959
3  0.493151       0.400000      0.000000           0.0        0.397959
4  0.150685       0.800000      0.000000           0.0        0.397959
5  0.273973       0.866667      0.000000           0.0        0.397959
6  0.438356       0.266667      0.000000           0.0        0.153061
7  0.479452       0.533333      0.000000           0.0        0.448980
8  0.191781       0.866667      0.829751           0.0        0.500000
9  0.342466       0.800000      0.742849           0.0        0.397959


### Data Discretization

### Data Discretization

#### Techniques Considered
The following techniques were evaluated for discretizing numerical data:

- **Binning**:  
  - Equal-width (distance) partitioning  
  - Equal-depth (frequency) partitioning  
- **History Analysis**

We choose **Equal-width Binning** for the following reasons:  
- It is simple and easy to implement manually.  
- It divides the data into intervals of equal width, which is suitable for columns like `age` and `hours-per-week`.  
- It does not require any assumptions about the data distribution.

#### Equal-width Binning Method

For a numerical column with a minimum value \(\text{min}\), a maximum value \(\text{max}\), and a specified number of bins \(k\):

- **Step 1: Calculate the Width of Each Bin**  
  The width of each bin is calculated using the formula:

  $$
  \text{width} = \frac{\text{max} - \text{min}}{k}
  $$


In [15]:
def equal_width_binning(column, num_bins, custom_bins=None, custom_labels=None):
    """
    Implement Equal-width Binning.
    
    Parameters:
    - column: Data column (pd.Series).
    - num_bins: Number of bins (intervals).
    - custom_bins: List of custom bin edges (if provided).
    - custom_labels: List of custom labels (if provided).
    
    Returns:
    - pd.Series containing the labels for each value.
    """
    # Check the case where all values are the same
    min_val = column.min()
    max_val = column.max()
    if max_val == min_val:
        label = custom_labels[0] if custom_labels else f"Bin_1"
        return pd.Series([label] * len(column))

    # If custom_bins is provided, use it
    if custom_bins:
        bins = custom_bins
        labels = custom_labels if custom_labels else [f"Bin_{i+1}" for i in range(len(bins)-1)]
    else:
        # Calculate equal-width intervals
        width = (max_val - min_val) / num_bins
        bins = [min_val + i * width for i in range(num_bins + 1)]
        labels = custom_labels if custom_labels else [f"Bin_{i+1}" for i in range(num_bins)]

    # Assign labels to each value
    return pd.cut(column, bins=bins, labels=labels, include_lowest=True)

**Step 2: Determine the Bin Boundaries**:
The boundaries of the bins are defined as:
$$
b_i = \text{min} + i \cdot \text{width}, \quad i = 0, 1, \ldots, k
$$
3. **Define the Intervals**:
   - Interval 1: $ [b_0, b_1] $
   - Interval 2: $(b_1, b_2]$
   - $ \ldots $
   - Interval $k$ :  $(b_{k-1}, b_k]$ 

**Step 4: Assign Labels to Values**:
   - If $x \in [b_i, b_{i+1})$  (or  $[b_0, b_1]$  for the first interval), then  $x$  is assigned the label  $\text{Bin}_{i+1}$ .
   - If $\text{max} = \text{min}$  all values are assigned the label $ \text{Bin}_1 $

In [16]:
# Define binning configurations for each numerical column
binning_configs = {
    'age': {
        # Bins in normalized range [0, 1]: [0, 0.2466, 0.5205, 1.0]
        # Corresponding to original values approximately [17, 35, 55, 90]
        # Labels: ['Young', 'Middle-aged', 'Senior']
        'bins': [0, 0.2466, 0.5205, 1.0],
        'labels': ['Young', 'Middle-aged', 'Senior']
    },
    'hours-per-week': {
        # Bins in normalized range [0, 1]: [0, 0.1939, 0.3979, 1.0]
        # Corresponding to original values approximately [1, 20, 40, 99]
        # Labels: ['Part-time', 'Full-time', 'Overtime']
        'bins': [0, 0.1939, 0.3979, 1.0],
        'labels': ['Part-time', 'Full-time', 'Overtime']
    },
    'education-num': {
        # Bins in normalized range [0, 1]: [0, 0.5333, 0.7333, 1.0]
        # Corresponding to original values approximately [1, 9, 12, 16]
        # Labels: ['Low', 'Medium', 'High']
        'bins': [0, 0.5333, 0.7333, 1.0],
        'labels': ['Low', 'Medium', 'High']
    },
    'capital-gain': {
        # Bins in normalized range [0, 1]: [-0.0001, 0, 1.0]
        # After Log Transformation + Min-Max Scaling, 0 corresponds to no gain
        # Labels: ['No Gain', 'Has Gain']
        'bins': [-0.0001, 0, 1.0],
        'labels': ['No Gain', 'Has Gain']
    },
    'capital-loss': {
        # Bins in normalized range [0, 1]: [-0.0001, 0, 1.0]
        # After Log Transformation + Min-Max Scaling, 0 corresponds to no loss
        # Labels: ['No Loss', 'Has Loss']
        'bins': [-0.0001, 0, 1.0],
        'labels': ['No Loss', 'Has Loss']
    }
}

Sử dụng hàm công thức và các khoảng mà ta định nghĩa lên tập.

In [17]:
# Apply Equal-width Binning for numeric columns
numeric_cols = ['age', 'education-num', 'capital-gain', 'capital-loss', 'hours-per-week']

for col in numeric_cols:
    config = binning_configs.get(col, {})
    custom_bins = config.get('bins')
    custom_labels = config.get('labels')
    
    # Apply Equal-width Binning
    df[f"{col}"] = equal_width_binning(
        df[col],
        num_bins=3,
        custom_bins=custom_bins,
        custom_labels=custom_labels
    )

df.head(10)

,age,education-num,relationship,occupation,capital-gain,capital-loss,hours-per-week,income
0,Middle-aged,High,Not-in-family,Adm-clerical,Has Gain,No Loss,Overtime,0
1,Middle-aged,High,Husband,Exec-managerial,No Gain,No Loss,Part-time,0
2,Middle-aged,Medium,Not-in-family,Handlers-cleaners,No Gain,No Loss,Overtime,0
3,Middle-aged,Low,Husband,Handlers-cleaners,No Gain,No Loss,Overtime,0
4,Young,High,Wife,Prof-specialty,No Gain,No Loss,Overtime,0
5,Middle-aged,High,Wife,Exec-managerial,No Gain,No Loss,Overtime,0
6,Middle-aged,Low,Not-in-family,Other-service,No Gain,No Loss,Part-time,0
7,Middle-aged,Medium,Husband,Exec-managerial,No Gain,No Loss,Overtime,1
8,Young,High,Not-in-family,Prof-specialty,Has Gain,No Loss,Overtime,1
9,Middle-aged,High,Husband,Exec-managerial,Has Gain,No Loss,Overtime,1


Preprocessing Data complete. Data written to 'adult_preprocessed.csv'

In [18]:
df.to_csv('adult_preprocessed.csv', index=False)